In [16]:
import scipy as sc
import numpy as np

In [17]:
#Matriz de adjacencia que modela o grafo cujo os vértices são as opiniões/posts
A =[[0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0]]

#PARAMETROS INICIAIS DO PROBLEMA

B = 0.67 #Taxa de transmissão

#Probabilidade de um post influenciar um vértice i num tempo t+1
def probx(tempo, MA, i, Taxainfo): 
    prod = 1
    x = [sc.stats.norm.pdf(t, 0, 1), sc.stats.poisson.pmf(t, 1), sc.stats.bernoulli.pmf(t, 1, 1), sc.stats. poisson.pmf(t, 1)] #vetor de estado
    xi = np.array(x)
    xi.transpose
    for j in range(len(A)):
        prod *= (1 - B*A[i][j]*xi[j]) #produtorio
    return prod

#Calculo de quanto tempo cada vertice leva para ser influenciado
for i in range(len(A)):
    prob = 0
    t = 0 #Parâmetros de informação
    k = 0 #Contador de iteração
    while prob != 1:
        prob = probx(t, A, i, B)
        if prob > 0.9999999999:
                prob = 1
        
        if prob == 1:
            print(f'vértice {i+1} foi convencido por t após {k} tempos')
            prob = 0
            break
        else:
            print(f'vértice {i+1} sucetivel a t com probabilidade atual de {prob} no tempo {t}')
        
        t = t + 1
        k = k+1
    



vértice 1 sucetivel a t com probabilidade atual de 0.7535207744151337 no tempo 0
vértice 1 sucetivel a t com probabilidade atual de 0.7535207744151337 no tempo 1
vértice 1 sucetivel a t com probabilidade atual de 0.8767603872075669 no tempo 2
vértice 1 sucetivel a t com probabilidade atual de 0.9589201290691889 no tempo 3
vértice 1 sucetivel a t com probabilidade atual de 0.9897300322672973 no tempo 4
vértice 1 sucetivel a t com probabilidade atual de 0.9979460064534594 no tempo 5
vértice 1 sucetivel a t com probabilidade atual de 0.9996576677422433 no tempo 6
vértice 1 sucetivel a t com probabilidade atual de 0.9999510953917491 no tempo 7
vértice 1 sucetivel a t com probabilidade atual de 0.9999938869239686 no tempo 8
vértice 1 sucetivel a t com probabilidade atual de 0.9999993207693298 no tempo 9
vértice 1 sucetivel a t com probabilidade atual de 0.9999999320769329 no tempo 10
vértice 1 sucetivel a t com probabilidade atual de 0.9999999938251757 no tempo 11
vértice 1 sucetivel a t co